# 0.0 Imports

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import pandas as pd
import numpy as np

## 0.1 Functions

In [ ]:
def soup(table):
    
    #cleaned_soup = BeautifulSoup( table, 'html.parser')
    cleaned_soup = str(table)
    cleaned_soup = cleaned_soup.split('\n')
    
    return cleaned_soup

def cleaning(table):
    
    #teams names
    
    #getting all upper cases
    only_uppercase = []
    only_uppercase.append([c for c in table if c.isupper()])
    only_uppercase = only_uppercase[0]
    
    #removing other strings
    element_1 = table[0][-4:] #removing dates
    element_2 = str('FT') #removing FT
    
    for element in only_uppercase:
        if element.endswith(element_1):
            only_uppercase.remove(element)
    
    for element in only_uppercase:
        if element.endswith(element_2):
            only_uppercase.remove(element)          
    
    #getting home and away teams
    home_teams = only_uppercase[0::2]
    away_teams = only_uppercase[1::2] 
    
    #home and away teams to lowercase
    home_teams = [x.lower() for x in home_teams]
    away_teams = [x.lower() for x in away_teams]
    
    
    #home and away goals
    scores = []
    scores.append([c for c in table if (' - ' in c) & (len(c) == 5)])
    scores = scores[0]
    
    home_goals = []
    away_goals = []

    for score in scores:
        home_goals.append(score[0])
        away_goals.append(score[-1])
    
    year = table[0][-4:]
    columns = ['home_teams', 'away_teams', 'home_goals', 'away_goals']
    
    df = pd.DataFrame(list(zip(home_teams, away_teams, home_goals, away_goals)), columns=columns)
    
    df['year'] = year
    
    return df

# 1.0 Webscrapping

In [244]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get( "https://www.fifa.com/tournaments/mens/worldcup/1994usa/match-center" )

driver.set_window_size(1024, 600)
driver.maximize_window()

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "onetrust-reject-all-handler"))
    )
    element.click()
    
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//span[contains(text(), 'Groups')]"))
    )
    
    time.sleep(2)
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
    time.sleep(2)
    element.click()   

    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, "fp-accordion-section_innerCards__2SOSW"))
    )
    
    fifa_1998_games = element.text
    
    driver.quit()  
except:
    driver.quit()

In [100]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)

driver.get( "https://www.fifa.com/tournaments/mens/worldcup/2018russia/match-center" )

driver.set_window_size(1024, 600)
driver.maximize_window()

time.sleep(2)
link = driver.find_element_by_id("onetrust-reject-all-handler")
link.click()

time.sleep(3)
link2 = driver.find_element_by_xpath("//span[contains(text(), 'Groups')]")
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", link2)
link2.click()

time.sleep(2)
games18 = driver.find_element(By.CLASS_NAME, "fp-accordion-section_innerCards__2SOSW")
fifa_2018_page1 = games18.text

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//span[contains(text(), 'Groups')]"}
  (Session info: chrome=107.0.5304.107)


# 2.0 Data Wrangling

In [245]:
fifa_94 = soup(fifa_1998_games)
df_94 = cleaning(fifa_94)

In [250]:
df = pd.concat([df_18, df_14, df_10, df_06, df_02, df_98, df_94]).reset_index(drop=True)

In [252]:
df['home_teams'].unique()

array(['russia', 'egypt', 'morocco', 'portugal', 'france', 'argentina',
       'peru', 'croatia', 'costa rica', 'germany', 'brazil', 'sweden',
       'belgium', 'tunisia', 'colombia', 'poland', 'uruguay', 'ir iran',
       'denmark', 'nigeria', 'serbia', 'korea republic', 'england',
       'japan', 'saudi arabia', 'spain', 'australia', 'iceland', 'mexico',
       'switzerland', 'senegal', 'panama', 'chile', "côte d'ivoire",
       'iran', 'ghana', 'cameroon', 'italy', 'honduras', 'united states',
       'netherlands', 'greece', 'bosnia-herzegovina', 'ecuador',
       'algeria', 'south africa', 'new zealand', 'slovenia', 'slovakia',
       'paraguay', 'korea dpr', 'trinidad and tobago',
       'serbia and montenegro', 'angola', 'czech republic', 'togo',
       'ukraine', 'republic of ireland', 'china pr', 'turkey',
       'yugoslavia', 'jamaica', 'romania', 'scotland', 'norway',
       'bulgaria', 'bolivia'], dtype=object)

In [253]:
df_18['home_teams'].unique()

array(['russia', 'egypt', 'morocco', 'portugal', 'france', 'argentina',
       'peru', 'croatia', 'costa rica', 'germany', 'brazil', 'sweden',
       'belgium', 'tunisia', 'colombia', 'poland', 'uruguay', 'ir iran',
       'denmark', 'nigeria', 'serbia', 'korea republic', 'england',
       'japan', 'saudi arabia', 'spain', 'australia', 'iceland', 'mexico',
       'switzerland', 'senegal', 'panama'], dtype=object)

In [256]:
df.to_csv('all_games_world_cup.csv')